In [12]:
#Instalamos las dependencias necesarias
#!pip install ultralytics roboflow
#!pip install wandb  # Para seguimiento de entrenamiento
from ultralytics import YOLO
import yaml
import os
from datetime import datetime
import time

# Función para formatear tiempo
def format_time(seconds):
    return time.strftime('%H:%M:%S', time.gmtime(seconds))


# Ruta al directorio de tu dataset
DATASET_PATH = 'train_data/dataset/logos'

# Crear configuración del dataset
data_config = {
    'path': DATASET_PATH,
    'train': 'train_data/dataset/logos/train/images',
    'val': 'train_data/dataset/logos/valid/images',
    'test': 'train_data/dataset/logos/test/images',
    'names': ['adidas', 'apple', 'ericsson', 'nike', 'reebok', 'samsung', 'sony']
}

# Imprimir información del dataset
print("\n=== CONFIGURACIÓN DEL DATASET ===")
for key, path in data_config.items():
    if key != 'names':
        n_images = len(os.listdir(path)) if os.path.exists(path) else 0
        print(f"{key}: {n_images} imágenes en {path}")
print(f"Clases: {data_config['names']}\n")


# Guardar la configuración
config_path = 'train_data/dataset/logos/data.yaml'
with open(config_path, 'w') as file:
    yaml.dump(data_config, file)

# Inicializar modelo
modelo = YOLO('yolov8n.pt')

# Configurar callbacks para métricas
def on_train_epoch_end(trainer):
    epoch = trainer.epoch
    metrics = trainer.metrics
    print(f"\nEpoch {epoch + 1}:")
    print(f"mAP50: {metrics['metrics/mAP50(B)']:.3f}")
    print(f"Precision: {metrics['metrics/precision(B)']:.3f}")
    print(f"Recall: {metrics['metrics/recall(B)']:.3f}")
    print(f"Tiempo transcurrido: {format_time(time.time() - start_time)}")

# Configurar hiperparámetros
hiperparametros = {
    'epochs': 150,
    'batch': 8,
    'imgsz': 640,
    'patience': 30,
    'optimizer': 'Adam',
    'lr0': 0.001,
    'lrf': 0.01,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 3,
    'cos_lr': True,
    'close_mosaic': 10,
    'label_smoothing': 0.1,
    'device': 'cpu',
    'augment': True,
    'workers': 1,
    'verbose': True
}

# Entrenar el modelo
print("\nHiperparámetros:")
for key, value in hiperparametros.items():
    if key != 'callbacks':
        print(f"{key}: {value}")

resultados = modelo.train(
    data=config_path,
    **hiperparametros
)

# Tiempo total de entrenamiento
training_time = time.time() - start_time
print(f"\nTiempo total de entrenamiento: {format_time(training_time)}")

# Evaluar el modelo
print("\n=== EVALUACIÓN DEL MODELO ===")
metricas = modelo.val()

# Resumen final
print("\n=== RESUMEN FINAL ===")
print(f"Mejor mAP50: {max(resultados.results_dict['metrics/mAP50(B)']):.3f}")
print(f"Mejor Precision: {max(resultados.results_dict['metrics/precision(B)']):.3f}")
print(f"Mejor Recall: {max(resultados.results_dict['metrics/recall(B)']):.3f}")

# Exportar el modelo
print("\n=== EXPORTANDO MODELO ===")
modelo.export(format='onnx')
modelo.export(format='torchscript')
print("Exportación completada!")


=== CONFIGURACIÓN DEL DATASET ===
path: 0 imágenes en train_data/dataset/logos
train: 0 imágenes en train_data/dataset/logos/train/images
val: 0 imágenes en train_data/dataset/logos/valid/images
test: 0 imágenes en train_data/dataset/logos/test/images
Clases: ['adidas', 'apple', 'ericsson', 'nike', 'reebok', 'samsung', 'sony']



FileNotFoundError: [Errno 2] No such file or directory: 'train_data/dataset/logos/data.yaml'